In [1]:
!pip install pyyaml h5py


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
import os
import tensorflow as tf
from tensorflow import keras

In [4]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [5]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
#użycie callbacks - punktów kontrolnych
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [7]:
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1510 - accuracy: 0.6754
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 4s 111ms/step - loss: 1.1464 - accuracy: 0.6760 - val_loss: 0.7228 - val_accuracy: 0.7830
Epoch 2/10
18/32 [===============>..............] - ETA: 0s - loss: 0.4647 - accuracy: 0.8733
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 66ms/step - loss: 0.4268 - accuracy: 0.8800 - val_loss: 0.5557 - val_accuracy: 0.8140
Epoch 3/10
18/32 [===============>..............] - ETA: 0s - loss: 0.2512 - accuracy: 0.9340
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 69ms/step - loss: 0.2841 - accuracy: 0.9230 - val_loss: 0.4719 - val_accuracy: 0.8430
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 0.2271 - accuracy: 0.9450
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 57ms/step - l

In [8]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [9]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model niewytrenowany: {acc}')

32/32 - 0s - loss: 2.4210 - accuracy: 0.0830 - 266ms/epoch - 8ms/step
model niewytrenowany: 0.08299999684095383


In [10]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z załadowanymi wagami: {acc}')

32/32 - 0s - loss: 0.4082 - accuracy: 0.8700 - 73ms/epoch - 2ms/step
model z załadowanymi wagami: 0.8700000047683716


In [11]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 32

In [12]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=5*batch_size)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

In [13]:
model.fit(train_images,
          train_labels,
          epochs=50,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback],
          batch_size=batch_size,
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [14]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0050.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001']

In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [16]:
model = create_model()
model.load_weights(latest)

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z załadowanymi wagami - latest: {acc}')

32/32 - 0s - loss: 0.5139 - accuracy: 0.8690 - 268ms/epoch - 8ms/step
model z załadowanymi wagami - latest: 0.8690000176429749


In [17]:
#zapis całego modelu
model = create_model()
model.fit(train_images,train_labels,epochs=5)

!mkdir -p saved_model
model.save('saved_model/mojmodel')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.1508 - accuracy: 0.6690
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4149 - accuracy: 0.8850
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2886 - accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2204 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1526 - accuracy: 0.9710
INFO:tensorflow:Assets written to: saved_model/mojmodel/assets


In [18]:
new_model = tf.keras.models.load_model('saved_model/mojmodel')
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss,acc = new_model.evaluate(test_images,test_labels,verbose=2)
print(f'model załadowany z archiwum: {acc}')

32/32 - 0s - loss: 0.4309 - accuracy: 0.8660 - 193ms/epoch - 6ms/step
model załadowany z archiwum: 0.8659999966621399


In [20]:
new_model.save('mmod.h5')

In [21]:
next_model = tf.keras.models.load_model('mmod.h5')
next_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
